In [1]:
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.saving import register_keras_serializable
from sklearn.model_selection import GridSearchCV


2025-03-23 13:01:00.267541: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742722260.282776   23317 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742722260.287662   23317 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-23 13:01:00.302854: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
california_housing = fetch_california_housing()

X_train0, X_test, Y_train0, Y_test = train_test_split(california_housing["data"], california_housing["target"])
X_train1, X_validation, Y_train1, Y_validation = train_test_split(X_train0, Y_train0)

sc = StandardScaler()
X_train_s = sc.fit_transform(X_train1)
X_validation_s = sc.fit_transform(X_validation)
X_test_s = sc.transform(X_test)


X_train_s_1, X_train_s_2 = X_train_s[:, :6], X_train_s[:, :-4]
X_validation_s_1, X_validation_s_2 = X_validation_s[:, :6], X_validation_s[:, :-4]
X_test_s_1, X_test_s_2 = X_test_s[:, :6], X_test_s[:, :-4]


@register_keras_serializable()
class WideAndDeepANN(keras.Model):
    def __init__(self, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden_layer_1 = keras.layers.Dense(50, activation=activation)
        self.hidden_layer_2 = keras.layers.Dense(10, activation=activation)
        self.final_output = keras.layers.Dense(1)
        self.helper_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_1, input_2 = inputs
        h1_out = self.hidden_layer_1(input_1)
        h2_out = self.hidden_layer_2(h1_out)
        concat_out = keras.layers.concatenate([input_2, h2_out])
        final_out = self.final_output(concat_out)
        helper_out = self.helper_output(h2_out)
        return final_out, helper_out


model_f = WideAndDeepANN()

model_checkpoint_callback = keras.callbacks.ModelCheckpoint("model_cb_regression_housing.keras", save_best_only=True)
early_stopping_callback = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
tb_callback = keras.callbacks.TensorBoard("tb_log3")

model_f.compile(loss=["mse", "mse"],
                loss_weights=[0.8, 0.2],
                optimizer="sgd",
                metrics=[["mean_absolute_error"], ["mean_absolute_error"]])

model_f.fit((X_train_s_1, X_train_s_2), (Y_train1, Y_train1), epochs=2,
            validation_data=((X_validation_s_1, X_validation_s_2), (Y_validation, Y_validation)),
            callbacks=[tb_callback])
model_f.save("housing_reg_model.keras")
model_f_reg = keras.models.load_model("housing_reg_model.keras")

class Mycallback(keras.callbacks.Callback):
    def on_train_end(self, logs=None):
        print("")

    def on_epoch_end(self, epoch, logs):
        print(logs["var_loss"])


mycb = Mycallback

def ann_model(number_of_hidden_layers=1,
              number_of_neurons=50,
              lr=0.01):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=[8]))
    for hidden_layer in range(number_of_hidden_layers):
        model.add(keras.layers.Dense(number_of_neurons, activation="selu"))
    model.add(keras.layers.Dense(1))
    sgd = keras.optimizers.SGD(lr=lr)
    model.compile(loss="mse", optimizer=sgd)
    return model

Epoch 1/2


W0000 00:00:1742722262.448116   23317 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.5176 - mean_absolute_error: 0.8231 - mean_absolute_error_1: 1.0053 - mse_loss: 1.9224 - val_loss: 0.5569 - val_mean_absolute_error: 0.5429 - val_mean_absolute_error_1: 0.5474 - val_mse_loss: 0.5591
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5800 - mean_absolute_error: 0.5550 - mean_absolute_error_1: 0.5604 - mse_loss: 0.5812 - val_loss: 0.5243 - val_mean_absolute_error: 0.5232 - val_mean_absolute_error_1: 0.5213 - val_mse_loss: 0.5252


In [3]:
# keras_sk_reg = keras.wrappers.scikit_learn.KerasRegressor(build_fn=ann_model)

In [4]:
keras_sk_reg = keras.wrappers.SKLearnRegressor(model=ann_model)

In [5]:
param_grid = {"model__number_of_hidden_layers": [1, 3, 5],
              "model__number_of_neurons": [50, 100, 150],
              "model__lr": [0.001, 0.01, 0.1]}

In [6]:
keras_sk_reg_gs = GridSearchCV(keras_sk_reg, param_grid)

In [7]:
keras_sk_reg_gs.fit(X_train_s, Y_train1, epochs=10,
                    validation_data=(X_validation, Y_validation),
                    callbacks=[keras.callbacks.EarlyStopping(patience=5)])

AttributeError: 'function' object has no attribute 'set_params'